In [1]:
from STL_to_path import *
from simple_GPT import *
from NL_to_STL import *

         Install drake (https://drake.mit.edu/installation.html)
         to use the Drake-based solvers.


In [2]:
T_max = 20          # time horizon in seconds 
dt = 0.5            # time step in seconds
N = int(T_max/dt)   # number of time steps
max_acc = 50        # maximum acceleration in m/s^2
max_speed = 1       # maximum speed in m/s

objects = {"key" : (3.75, 4.75, 3.75, 4.75, 1., 2.),
           "chest": (-4.25, -3, -4.5, -3.75, 0., 0.75),
           "door": (0., 0.5, -2.5, -1, 0., 2.5),
           "bounds": (-5., 5., -5., 5., 0., 3.),
           "NE_inside_wall": (2., 5., 3., 3.5, 0., 3.),
           "south_mid_inside_wall": (0., 0.5, -5., -2.5, 0., 3.),
           "north_mid_inside_wall": (0., 0.5, -1., 5., 0., 3.),
           "west_inside_wall": (-2.25, -1.75, -5., 3.5, 0., 3.),
           "top_door": (0., 0.5, -2.5, -1, 2.5, 3.),
           }

x0 = np.array([0.,0.,0.,0.,0.,0.]) # initial state: x, y, z, vx, vy, vz

In [3]:
translator = NL_to_STL(objects, T_max, dt, print_instructions=False)
gpt = GPT()

In [4]:
messages = []
raw_system_message = translator.load_chatgpt_instructions()
system_message = translator.insert_instruction_variables(raw_system_message)
messages.append({"role": "system", "content": system_message})

In [5]:
print("Assistant:", system_message, "\n", "______________________________")
i=0
while i<10:
    user_input = input("You: ")
    print("User:", user_input)

    if user_input.lower() == 'quit' or user_input.lower() == 'exit':
        print("Goodbye!")
        break

    messages.append({"role": "user", "content": user_input})
    response = gpt.chatcompletion(messages)
    print("Assistant:", response)
    messages.append({"role": "assistant", "content": response})

    # check if < or > symbol is present in the response
    if '<' in response:
        print("< detected")
        break

    i+=1

Assistant: You are an assistant guiding a drone to perform an instruction specified by a user.
Your task is to take the user's instruction and translate it into a specification in Signal Temporal Logic (STL).
You are provided with a dictionary of objects in the scene, the available Logic functions, and Logic operators that you can use in your generated STL specification. All of these are provided below.

Objects:
{'key': (3.75, 4.75, 3.75, 4.75, 1.0, 2.0), 'chest': (-4.25, -3, -4.5, -3.75, 0.0, 0.75), 'door': (0.0, 0.5, -2.5, -1, 0.0, 2.5), 'bounds': (-5.0, 5.0, -5.0, 5.0, 0.0, 3.0), 'NE_inside_wall': (2.0, 5.0, 3.0, 3.5, 0.0, 3.0), 'south_mid_inside_wall': (0.0, 0.5, -5.0, -2.5, 0.0, 3.0), 'north_mid_inside_wall': (0.0, 0.5, -1.0, 5.0, 0.0, 3.0), 'west_inside_wall': (-2.25, -1.75, -5.0, 3.5, 0.0, 3.0), 'top_door': (0.0, 0.5, -2.5, -1, 2.5, 3.0)}

Logic functions: 
"STL_formulas.inside_cuboid(object)", "STL_formulas.outside_cuboid(object)"

Logic operators:
"&", "|"

STL specifications

In [6]:
final_response = messages[-1]["content"]
final_response

'<STL_formulas.inside_cuboid(objects["chest"]).eventually(0, 20) & STL_formulas.inside_cuboid(objects["chest"]).always(0, 20)>'

In [8]:
def extract_specs(response):
    response = response
    specs = []
    start = 0
    while True:
        start = response.find("<", start)
        if start == -1:
            break
        end = response.find(">", start)
        specs.append(response[start+1:end])
        start = end
    return specs

In [10]:
specs = extract_specs(final_response)
print(specs)

['STL_formulas.inside_cuboid(objects["chest"]).eventually(0, 20) & STL_formulas.inside_cuboid(objects["chest"]).always(0, 20)']
